# Storage-Blob
* 파이썬에서 blob을 사용하는 방법에 대해 다루어 본다.

In [ ]:
!pip install azure-storage-blob

In [2]:
# 필요한 패키지를 불러옵니다.

import os, uuid # uuid를 사용해서 임시 패키지를 사용한다.
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient # 사용할 blob 패키지를 불러온다.

In [3]:
# Blob 서비스에 연결합니다.
connect_string = '' # 주소랑 키 값등이 필요하다. 그걸 connect_string으로 만든다.
# 애저 스토리지 계정에서 액세스 키에서 첫번째 키의 연결 문자열을 복사한다.

# 아래 코드는 블랍 스트링을 하는 서비스가 만들어진다.
blob_service_client = BlobServiceClient.from_connection_string(connect_string) # 얘를 이용해서 사용한다.

In [ ]:
# Container 생성한다.
container_name = str(uuid.uuid4()) # 겹치지 않는 문자열을 만들 때 사용한다.
print(container_name)

container_client = blob_service_client.create_container(container_name) # 실제로 컨테이너를 만드는 방법 그리고 위에서 난수로 이름을 만들었던 것을 적용해준다.

* 컨테이너 이름을 매번 설정할 수 없어서 난수로 만들어 준다.
* 애저 스토리지 계정에서의 컨테이너에 가보면 생성이 되어있는 것을 볼 수 있다.

In [8]:
# 데이터를 업로드 한다.
local_path = './data' # 로컬의 경로를 잡아본다. 데이터 폴터 밑에 있는
os.mkdir(local_path) # 업로드할 파일이 저장될 위치를 만들어준다.

local_file_name = str(uuid.uuid4()) + '.txt' # 파일의 이름도 서로 중복이 되면 안되서 문자열로 만들어 준다.
upload_file_path = os.path.join(local_path, local_file_name) # 조인을 사용해서 두개의 경로를 합쳐준다.

* 컨테이너에 직접 파일을 업로드를 한다.

In [9]:
# 업로드 할 파일을 준비한다.
file = open(file = upload_file_path, mode = 'w') # 모드는 쓰기모드로 열어준다.
file.write('Hello Azure Storage')
file.close()  # 파일을 썼으면 닫아준다.

# 이렇게 하면 컨테이너와 파일이 준비가 되었다.

blob_client = blob_service_client.get_blob_client( # blob 클라이언트를 만들어 준다. blob클라이언트를 만들어 줘 하고 만들어준다.
                                  container = container_name,
                                  blob = local_file_name) # local file name이 최종 경로이다.

# 파일 업로드
with open(file = upload_file_path, mode = 'rb') as data:  # 모드는 바이너리 단위로 텍스트를 읽어들인다. 준비가 되어 있는 파일 경로를 써준다. 파일을 읽어서 데이터라고 해준다.
  blob_client.upload_blob(data)  # 업로드 블랍이라고 써준다.

* 여기까지 했으면 파일을 업로드 하는 것이다.
* 컨테이너에서 보게 되면 확장자가 txt로 바뀌는 것을 볼 수 있다.
* 그래서 파일이 저장되어 있는 것을 볼 수 있다.
* 애저의 컨테이너에 들어가서 액세스를 컨테이너로 바꾸어 주게 되면 주소를 웹에 입력시키면 저장을 시킬 수 있다.


In [ ]:
# 파일의 목록의 확인
blob_list = container_client.list_blobs() # 업로드되 블랍의 리스를 가지고 올 수 있다.
for blob in blob_list:
  print('\t' + blob.name) # 파일의 이름이 잡힌 것을 볼 수 있다.

In [12]:
# 업로드된 파일의 다운로드
dowonload_file_path = os.path.join(local_path, # local의 path를 지정한다. 
                                   str.replace(local_file_name, '.txt', 'DOWNLOAD.txt')) # replace로 해서 만들어 준다.
container_client = blob_service_client.get_container_client(container = container_name) # 컨테이너의 이름이 중요하다. 그리고 컨테이너를 다운로드 받아준다. 업로드 하는 쪽이랑 다운로드 하는 쪽이랑 주소가 다를 수 있어서 만들어 준다.

with open(file = dowonload_file_path, mode = 'wb') as download_file: # 우리가 여기에다가 파일을 다운로드 받아준다.
  download_file.write(container_client.download_blob(blob.name).readall()) # 컨테이너 클라이언트한테, blob을 다운로드 받아들이게 해준다. 그리고 이름을 지정해준다. 그리고 전체 데이터를 다운로드 받아준다.

* 그럼 파일에 다운로드 점 텍스트 파일로 해서 만들어져 있는 것을 볼 수 있다.

In [13]:
# 실습한 자원의 정리
print('Press the Enter key to begin clean up') # 메세지를 하나 준다.
input() # 그리고 하나씩 지워준다. 엔터를 누르면 삭제가 된다.

print('Deleting blob container...')
container_client.delete_container() # 컨테이너를 지우면 컨테이너 안에 있는게 모두 지워진다.

print('Deleting the local source and downloaded files...')
os.remove(upload_file_path) # 파일도 지워준다.
os.remove(dowonload_file_path)
os.rmdir(local_path)

print('Done')

Press the Enter key to begin clean up

Deleting blob container...
Deleting the local source and downloaded files...
Done


* 만들었던 작업들을 정리를 해본다.
* 가장 많이 사용하는 스토리지이다.
* blob에 집어넣으면 Azure CDN